# Práctica 1. Levantar un Servidor Local de LLM y realizar una tarea básica utilizando su API

El programa procesa facturas en formato PDF a un archivo CSV que sirve como resumen del total de las facturas. Mediante el uso de un servidor local de LM Studio y un LLM, se consigue completar esta tarea.

#### Fuentes utilizadas

- Listar archivos de un directorio: https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
  
- Interacción con los modelos LLM de LM Studio (se sigue el ejemplo que se encuentra en la sección _Endpoints overview_): https://lmstudio.ai/docs/app/api/endpoints/openai

  
- Parámetros disponibles en la interacción con los modelos LLM: https://platform.openai.com/docs/api-reference/chat/create

In [6]:
from PyPDF2 import PdfReader
from openai import OpenAI
import pandas as pd
from os import listdir
from os.path import isfile, join
import sys

In [7]:
# Ruta del directorio donde están las facturas.
DIRECTORIO_FACTURAS = "./facturas/"

# Ruta del almacenamiento del CSV generado
RUTA_CSV = "./resumenes/resumen_facturas.csv"

# Separador utilizado para los campos del CSV.
SEPARADOR = ";"
SEPARADOR_NOMBRE = "semicolon"

In [8]:
# Temperatura para la creatividad del modelo.
TEMPERATURA = 0.4

# Nivel de esfuerzo que debe aplicar al razonar.
RAZONAMIENTO = "minimal"

# Prompt para el modelo.
PROMPT = f"""You are going to receive an invoice data in text format. Read it and identify the following three fields on the invoice:
                1. Customer
                2. Date
                3. Total amount
            Once identified return the three fields separated by {SEPARADOR_NOMBRE} in the following format:
                Customer{SEPARADOR} (DD-MM-YYYY){SEPARADOR} Total Amount
            For example:
                Enterprise, S.L.{SEPARADOR} (01-11-2025){SEPARADOR} 40
            Remember, return only these three items in the format indicated (separated by {SEPARADOR_NOMBRE} on a single line) and nothing else.
            The default values for these fields are:
                None{SEPARADOR} (00-00-0000){SEPARADOR} 0
            Use them if you do not find any information required.
            For decimal numbers use commas instead of dots (Example: 40,0 instead of 40.0)
            Include only the amount, not the currency.
         """

In [9]:
# Obtención de todos los archivos del directorio, no se consideran subdirectorios.
archivos = [archivo for archivo in listdir(DIRECTORIO_FACTURAS) if isfile(join(DIRECTORIO_FACTURAS, archivo))]

# Si no hay archivos, no se debe seguir.
if len(archivos) == 0:
    print("No hay facturas.")
    sys.exit(0) # Salir del programa.

In [10]:
# Acceso al modelo.
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
MODELO_LLM = "phi-3.1-mini-4k-instruct"

respuestas = { "Customer" : [], "Date" : [], "Total_Amount" : []}

# Procesamiento de cada factura.
for factura in archivos:
    
    lector = PdfReader(f"{DIRECTORIO_FACTURAS}{factura}")
    texto = ""

    # Se pasa a texto cada página del PDF.
    for pagina in lector.pages:
        texto += pagina.extract_text()

    # Interacción con el modelo.
    completion = client.chat.completions.create(
        model = MODELO_LLM,
        temperature = TEMPERATURA,
        reasoning_effort = RAZONAMIENTO,
        messages = [
            { "role": "system", "content": PROMPT },
            { "role": "user", "content": texto }
        ]
    )

    # Respuesta del modelo.
    contenido = completion.choices[0].message.content
    cliente, fecha, total = contenido.split(SEPARADOR, maxsplit = 2)

    # Adición de la respuesta.
    respuestas["Customer"].append(cliente.strip())
    respuestas["Date"].append(fecha.strip())
    respuestas["Total_Amount"].append(total.strip())

# Creación del CSV.
df = pd.DataFrame(respuestas)
df.to_csv(RUTA_CSV, index=False, sep=SEPARADOR, encoding="utf-8-sig")

print(f"Resúmenes generados en {RUTA_CSV}")

Resúmenes generados en ./resumenes/resumen_facturas.csv
